# Stock Price Prediction with LSTM (Long Short-Term Memory) Neural Network 

In [92]:
import pandas as pd
import yfinance as yf
import datetime
from datetime import date, timedelta

In [93]:
today = date.today()

d1 = today.strftime('%Y-%m-%d')
end_date = d1

d2 = date.today() - timedelta(days=7000)
d2 = d2.strftime('%Y-%m-%d')
start_date = d2

# print(start_date)
data = yf.download('AAPL',
                   start=start_date,
                   end=end_date,
                   progress=False,
                   auto_adjust=False)

# data = data.reset_index()
# print(data)

# data = [['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]
# data.columns = ['date', 'open', 'high', 'low', 'close', 'adj close', 'volume']
# print(data)

data['Date'] = data.index

data = data[['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]
data.reset_index(drop=True, inplace=True)
data.columns = data.columns.get_level_values(0)

# export dataset
# data.to_csv('dataset/new_dataset.csv', index=False)
# dataframe = pd.read_csv('dataset/new_dataset.csv')

In [94]:
print(data.tail())

Price       Date        Open        High         Low       Close   Adj Close  \
4816  2025-09-25  253.210007  257.170013  251.710007  256.869995  256.869995   
4817  2025-09-26  254.100006  257.600006  253.779999  255.460007  255.460007   
4818  2025-09-29  254.559998  255.000000  253.009995  254.429993  254.429993   
4819  2025-09-30  254.860001  255.919998  253.110001  254.630005  254.630005   
4820  2025-10-01  255.039993  258.790009  254.929993  255.449997  255.449997   

Price    Volume  
4816   55202100  
4817   46076300  
4818   40127700  
4819   37704300  
4820   48667300  


In [95]:
import plotly.graph_objects as go


figure = go.Figure(data=[go.Candlestick(x = data['Date'],
                                        open = data['Open'],
                                        high = data['High'],
                                        low = data['Low'],
                                        close = data['Close'])])

figure.show()

In [96]:
correlation = data.corr()
print(correlation['Close'].sort_values(ascending=False))

Price
Close        1.000000
Adj Close    0.999916
High         0.999911
Low          0.999906
Open         0.999805
Date         0.884979
Volume      -0.541301
Name: Close, dtype: float64


In [97]:
x = data[['Open', 'High', 'Low', 'Volume']]
x = x.to_numpy()

y = data[['Close']]
y = y.to_numpy()
# print(type(y))
y = y.reshape(-1, 1)
# print(type(y))

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)

In [98]:
from keras.models import Sequential
from keras.layers import Dense, LSTM

model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (x_train.shape[1], 1)))
model.add(LSTM(64, return_sequences=False))

model.add(Dense(25))
model.add(Dense(1))

model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_5 (LSTM)                   │ (None, 4, 128)         │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 25)             │         1,625 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 117,619 (459.45 KB)

 Trainable params: 117,619 (459.45 KB)

 Non-trainable params: 0 (0.00 B)

In [99]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, batch_size=1, epochs=30)

Epoch 1/30
3856/3856 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - loss: 387.8394
Epoch 2/30
3856/3856 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 33.6059
Epoch 3/30
3856/3856 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 23.2748
Epoch 4/30
3856/3856 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 20.7234
Epoch 5/30
3856/3856 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 16.6385
Epoch 6/30
3856/3856 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 16.3625
Epoch 7/30
3856/3856 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 13.2524
Epoch 8/30
3856/3856 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 15.0077
Epoch 9/30
3856/3856 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 11.9003
Epoch 10/30
3856/3856 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 10.2780
Epoch 11/30
3856/3856 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 10.5998
Epoch 12/30
3856/3856 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 11.5812
Epoch 13/30
3856/3856 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 13.3223
Epoch 14/30
3856/3856 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 10.0268
Epoch 15/30
3

In [100]:
import numpy as np
# features = Open, High, Low, Low, Adj Close, Volume
fitur = np.array([[180.086665, 185.980003, 178.330990, 82983342]])
model.predict(fitur)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step


array([[179.50487]], dtype=float32)